In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
from tqdm import tqdm

In [3]:
def create_search(keyword):
    cookies = {
        'advanced-frontend': '1qvup17bdkenkns7ja4ae37m0h',
        '_csrf-frontend': '336089dc6274e73228ccf35d519826f5d3bf03f31b26791424cedbd04e5f4472a%3A2%3A%7Bi%3A0%3Bs%3A14%3A%22_csrf-frontend%22%3Bi%3A1%3Bs%3A32%3A%22Veb1gE9jjqODCQlIEHn7wwZA7shoodp3%22%3B%7D',
        '_gid': 'GA1.3.807119358.1683069145',
        '_ga': 'GA1.3.67003413.1663762523',
        '_ga_3WFF0VD4BK': 'GS1.1.1683069145.12.1.1683069536.0.0.0',
    }

    headers = {
        'authority': 'mdar.mda.gov.my',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        # 'cookie': 'advanced-frontend=1qvup17bdkenkns7ja4ae37m0h; _csrf-frontend=336089dc6274e73228ccf35d519826f5d3bf03f31b26791424cedbd04e5f4472a%3A2%3A%7Bi%3A0%3Bs%3A14%3A%22_csrf-frontend%22%3Bi%3A1%3Bs%3A32%3A%22Veb1gE9jjqODCQlIEHn7wwZA7shoodp3%22%3B%7D; _gid=GA1.3.807119358.1683069145; _ga=GA1.3.67003413.1663762523; _ga_3WFF0VD4BK=GS1.1.1683069145.12.1.1683069536.0.0.0',
        'referer': 'https://mdar.mda.gov.my/frontend/web/index.php?r=carian',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    }

    params = {
        'r': 'carian/index',
        'CompletedApplicationAllmdrSearch[globalSearch]': keyword,
    }

    response = requests.get('https://mdar.mda.gov.my/frontend/web/index.php', params=params, cookies=cookies, headers=headers)
    soup = BeautifulSoup(response.text)
    device_list = soup.find_all('tr')
    return device_list
    

In [3]:
# #Load Keyword file
# keyword_file = pd.ExcelFile("IVD_Keyword_Tracker_v0.1.xlsx")
# player_names = pd.read_excel(keyword_file, sheet_name = 'IVD Final Keyword List')
# player_names = set(player_names[player_names['Player Name (2)'].notnull()]['Player Name (2)'])
# player_names

{'Abbott',
 'Agilent',
 'BD',
 'BIO-RAD',
 'BIOMERIEUX',
 'Cepheid',
 'Diasorin',
 'Grifols',
 'Hologic',
 'Horiba',
 'Human',
 'Leica',
 'Merck',
 'Mindray',
 'Qiagen',
 'Roche',
 'Siemens',
 'Sysmex',
 'Thermo Fisher'}

In [4]:
## Use IVD Gateway Major player to search MDA using company names
major_players = pd.read_excel("IVD_gateway_major_players_actual.xlsx")
major_players 

,production_cycle,study_year,player_code,player_name
0,2023,2022,PY0196,Roche
1,2023,2022,PY0006,Abbott
2,2023,2022,PY0064,Danaher - Beckman Coulter
3,2023,2022,PY0065,Danaher - Cepheid
4,2023,2022,PY0066,Danaher - Leica
...,...,...,...,...
327,2023,2022,PY9995,Boule Diagnostics
328,2023,2022,PY0375,Fokus Diagnostic
329,2023,2022,PY0397,Bioptik Technology
330,2023,2022,PY0338,Inbios


In [5]:
players = major_players.player_name.values.tolist()
## Split out company names delimited by '-'
keywords = []
for name in players:
    if '-' in name:
        spread_names = name.split('-')
        for i in spread_names:
            keywords.append(i.strip())
    else:
        keywords.append(name)

In [6]:
len(set(keywords))
keywords = list(set(keywords))

In [7]:
print("Searching MDA using keywords...")
catch = dict()
for keyword in keywords:
    print(keyword, end=" ")
    try:
        search_result = create_search(keyword)
        search_result = search_result[1:]
        if 'No results found.' in search_result[0].text:
            print("No Results")
        else:
            ids_in_results = [x['data-key'] for x in search_result]
            catch[keyword] =  ids_in_results
            print(len(ids_in_results))
    except:
        print("Error")
        
    
    

Searching MDA using keywords...
Seegene 11
Snibe 47
Philips 198
Abaxis No Results
Motic 1
Acon Diagnostics No Results
Mylab 6
Aus Diagnostics No Results
Promega No Results
MGI Tech No Results
Grifols 23
Sensa Core No Results
Tosoh 9
Aesku 9
TAUNS No Results
Liferiver 4
DiaMed 5
Transasia No Results
Diagon 1
Homebrew No Results
Bioneer 3
Abdulla Fouad Group No Results
IBP 25
Leica 57
Erba 8
Wuhan Easy Diagnostics No Results
Sakura 21
SureScreen No Results
General Biologicals No Results
CTK Biotech 1
Twist Bioscience No Results
Slee 88
Sysmex 82
Asan 41
Pathisto No Results
HiMedia 2
Muto 1
Infopia 1
Robonik 2
Premier Medical No Results
Rad 956
Kyowa No Results
Bioer 2
Diagnostic BioSystems No Results
VivaChek Biotech (Hangzhou) Co.,Ltd No Results
Acon Labs No Results
Kogene Biotech No Results
Toshiba No Results
Avantor No Results
YD 361
Kehua No Results
Fukuda Denshi No Results
Others No Results
EuroImmun 34
Immuno concepts 6
ACCB Biotech No Results
Aldrich 2
CapitalBio No Results
Nichir

In [10]:
import pickle
## store the dict for later.
with open(f'MDA_MY_MDR_Company_Records_110124.pkl', 'wb') as f:
    pickle.dump(catch, f)


In [12]:
ids_to_scrape = []
for k,v in catch.items():
    ids_to_scrape+= [*v]

ids_to_scrape  = list(set(ids_to_scrape))
len(ids_to_scrape)
    

17756

In [34]:
# Start actual data collection here
import os
import time
import requests
from requests.adapters import HTTPAdapter, Retry
from bs4 import BeautifulSoup
import pandas as pd
import warnings as ws
from datetime import datetime as date
import concurrent.futures
from tqdm import tqdm

# Constants
BASE_URL = "https://mdar.mda.gov.my/frontend/web/index.php?r=carian%2Fview&id="
MAX_RETRIES = 5
RETRY_INTERVAL = 1

# To ensure consistent directory creation without errors
current_directory = os.getcwd()
output_directory = os.path.join(current_directory, "Fetched Data")

if not os.path.exists(output_directory):
    os.mkdir(output_directory)

ws.filterwarnings("ignore")

def clean_str(string):
    return string.replace('\xa0', " ").strip()

def create_retry_session():
    retry_strategy = Retry(
        total=MAX_RETRIES,
        backoff_factor=RETRY_INTERVAL,
        status_forcelist=[500, 502, 503, 504],  # Retry on these HTTP status codes
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session = requests.Session()
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def fetch_data_with_retry(session, url):
    retries = 0
    while retries < MAX_RETRIES:
        try:
            response = session.get(url)
            response.raise_for_status()
            return response.text
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from {url}: {e}. Retrying in {RETRY_INTERVAL} seconds...")
            retries += 1
            time.sleep(RETRY_INTERVAL)

    print(f"Max retries reached. Unable to fetch data from {url}.")
    return None

def fetch_base_data(session, guid):
    create_url = BASE_URL + guid
    response_text = fetch_data_with_retry(session, create_url)
    if not response_text:
        return None

    soup = BeautifulSoup(response_text, 'html.parser')
    table1 = soup.find('table', id="w0")
    table2 = soup.find('table', id="example2")

    table1 = pd.read_html(str(table1))[0].transpose()
    table1.columns = table1.iloc[0].values
    table1 = table1.iloc[1:]
   
    table2 = pd.read_html(str(table2))[0]
    
    table1['sourceActual'] = create_url
    table1['gatewayCaptureAt'] = date.today()
    table2['sourceActual'] = create_url
    table2['gatewayCaptureAt'] = date.today()
    return table1, table2



def process_url(guid, session):
    device_list, device_identifier = fetch_base_data(session, guid)
    return device_list, device_identifier

def main():
    device_list_collector = pd.DataFrame()
    device_identifier_collector = pd.DataFrame()
    with create_retry_session() as session:
        with concurrent.futures.ThreadPoolExecutor(max_workers = 16) as executor:
            futures = [executor.submit(process_url, ids, session) for ids in ids_to_scrape]

            for future in tqdm(concurrent.futures.as_completed(futures), total=len(ids_to_scrape),
                               desc=f"Fetching data"):
                device_list, device_identifier = future.result()
                device_list_collector = pd.concat([device_list_collector, device_list])
                device_identifier_collector = pd.concat([device_identifier_collector, device_identifier])
    print("Saving files")
    device_list_collector.to_excel(f'Fetched Data/MDA_MY - Device list_110124.xlsx')
    device_identifier_collector.to_excel(f'Fetched Data/MDA_MY - Device Identifiers_110124.xlsx')

if __name__ == '__main__':
    main()


Fetching data: 100%|████████████████████████████████████████████████████████████▉| 17755/17756 [24:56<00:00,  7.83it/s]

Error fetching data from https://mdar.mda.gov.my/frontend/web/index.php?r=carian%2Fview&id=74775: HTTPSConnectionPool(host='mdar.mda.gov.my', port=443): Max retries exceeded with url: /frontend/web/index.php?r=carian%2Fview&id=74775 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000111C38518E0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')). Retrying in 1 seconds...
Error fetching data from https://mdar.mda.gov.my/frontend/web/index.php?r=carian%2Fview&id=74775: HTTPSConnectionPool(host='mdar.mda.gov.my', port=443): Max retries exceeded with url: /frontend/web/index.php?r=carian%2Fview&id=74775 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000111C9C5A070>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')). Retrying in 1 seconds...
Error fetching data from https://mdar.mda.gov.my/frontend/web/index.php?r=carian%2Fview&id=74775: HTTPSConnectionPool(host='mdar.mda.gov.m

Fetching data: 100%|███████████████████████████████████████████████████████████| 17756/17756 [1:02:47<00:00,  4.71it/s]
